# Nexthink AI Software Engineer assignment


First, we thank you for the time and effort you will invest in this assignment. We believe that this exercise will help us understand your technical skills and expertise in the field of artifical intelligence and machine learning.

You have seven days to complete the assignment. We estimate that the work required should take only a few hours at most. A jupyter notebook is provided as a starting point, but you are free to use any other format.

Please pay attention to the code and the description of your approach and results. The code should be well-documented and easy to understand. Additionally, we expect that you will provide a clear description of your approach to the problem and the rationale behind your methodology. In case of doubt, you can take any assumption you think is reasonable but make sure you explain it well. Lastly, we encourage you to present your results in an easy-to-understand manner, including visualizations and other appropriate metrics. 

We wish you the best of luck in completing the exercises, and we look forward to reviewing your submission. 

## 1. Uncovering topics behind news articles

*You are a tech-savvy journalist tasked with classifying news articles into categories. To save time, you decide to use your machine learning skills to automate this process.*

### Dataset

We provide a dataset (`news.jsonl`) containing around 210k news headlines between 2012 and 2022 from HuffPost. It contains the following attributes:

- `link`: link to the original news article.
- `headline`: the headline of the news article.
- `category`: category in which the article was published.
- `short_description`: Abstract of the news article.
- `authors`: list of authors who contributed to the article.
- `date`: publication date of the article.

The objective is to automatically determine the categories behind news articles. You will solely make use of the `headline` attribute **OR** the `short_description` attribute.

> For both parts below we expect you to explain and show that your solution works as expected (e.g., through metrics on a test dataset)

In [2]:
import pandas as pd
news_df = pd.read_json("data/news.jsonl", lines=True)
news_df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


### Known categories

You will first assume that you know the categories (e.g., the unique values of the `category` attribute). Train a model able to correctly classify the headline or description of news articles into the correct category.

#### SOLUTION: 
#### For this solution XGBOOST model is used after reducing the number of categories by normalizing them. 
#### The total number of remaining categories are 25.
#### With tfidf features extraction using with xgboost we get 0.48 as the weighted average F1 score. 
#### It is a good initial score considering the variance and the number of categories.
#### Transformer based techniques can have better performance in cases like these.
#### XGBOOST was chosen as it provides inherent class weights adjustment and is also comparatively robust to imbalance.

In [48]:
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words, wordnet
import xgboost as xgb
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


# Downloading necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /home/om/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/om/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/om/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
def preprocess_text(text):
    """A simple preprocessing utility"""
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing Stop Words and Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords.words('english')]
    
    # Removing punctuation
    tokens = [word for word in tokens if word.isalpha()]
    
    # Convert tokens back to string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [10]:
# Preprocess news_df short description
news_df["short_desc_preprocessed"] = news_df["short_description"].apply(lambda x:preprocess_text(x)) 

In [13]:
news_df = news_df[news_df['short_description'] != '']

In [15]:
# Apply the category normalization to combine categories of same topics

# 1. Combine "GREEN" and "ENVIRONMENT" into "ENVIRONMENT"
news_df['category'] = news_df['category'].replace(['GREEN', 'ENVIRONMENT'], 'ENVIRONMENT')

# 2. Combine "QUEER VOICES", "BLACK VOICES", and "LATINO VOICES" into "DIVERSE VOICES"
news_df['category'] = news_df['category'].replace(['QUEER VOICES', 'BLACK VOICES', 'LATINO VOICES'], 'DIVERSE VOICES')

# 3. Combine "WORLDPOST" and "THE WORLDPOST" and "U.S NEWS" into "WORLD NEWS". 
# note: Just an experiment considering that US News has US events which are sometime similar to world events
news_df['category'] = news_df['category'].replace(['WORLDPOST', 'THE WORLDPOST','U.S. NEWS'], 'WORLD NEWS')

# 4. Combine "PARENTING" and "PARENTS" into "PARENTING"
news_df['category'] = news_df['category'].replace(['PARENTING', 'PARENTS'], 'PARENTING')

# 5. Combine "ARTS", "ARTS & CULTURE", and "CULTURE & ARTS" into "ARTS & CULTURE"
news_df['category'] = news_df['category'].replace(['ARTS', 'ARTS & CULTURE', 'CULTURE & ARTS'], 'ARTS & CULTURE')

# 6. Combine "TECH" and "SCIENCE" into "TECH & SCIENCE"
news_df['category'] = news_df['category'].replace(['TECH', 'SCIENCE'], 'TECH & SCIENCE')

# 7. Combine "TASTE" and "FOOD & DRINK" into "FOOD & DRINK"
news_df['category'] = news_df['category'].replace(['TASTE', 'FOOD & DRINK'], 'FOOD & DRINK')

# 8. Combine "MONEY", "BUSINESS" into "BUSINESS & FINANCE"
news_df['category'] = news_df['category'].replace(['MONEY', 'BUSINESS'], 'BUSINESS & FINANCE')

# 9. Combine "COLLEGE", "EDUCATION" into "EDUCATION"
news_df['category'] = news_df['category'].replace(['COLLEGE', 'EDUCATION'], 'EDUCATION')

# 10. Combine "STYLE", "STYLE AND BEAUTY" into "STYLE & BEAUTY"
news_df['category'] = news_df['category'].replace(['STYLE', 'STYLE & BEAUTY', 'STYLE AND BEAUTY'], 'STYLE & BEAUTY')

# 11. Combine "GREEN" and "ENVIRONMENT" into "ENVIRONMENT"
news_df['category'] = news_df['category'].replace(['WELLNESS', 'HEALTHY LIVING'], 'WELLNESS')

# 12. Combine "WEIRD NEWS", "GOOD NEWS", "FIFTY" into "MISCELLANEOUS"
news_df['category'] = news_df['category'].replace(['WEIRD NEWS', 'GOOD NEWS', 'FIFTY'], 'MISCELLANEOUS')

In [23]:
# Splitting the data
label_encoder = LabelEncoder()
label_encoder.fit(news_df['category'].unique())
X = news_df['short_desc_preprocessed']
y = label_encoder.transform(news_df['category'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [41]:
# 1. Preprocessing and Tokenization
tfidf = TfidfVectorizer(max_features=10000, stop_words='english')
X_tfidf_train = tfidf.fit_transform(X_train)
X_tfidf_test = tfidf.transform(X_test)

In [42]:
# 2. Compute class weights based on median
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train) # "Balanced" is initialisation The next step changes the weights
class_weights_dict = dict(enumerate(class_weights))
weights = [class_weights_dict[label] for label in y_train]

# Adjust weights based on median of all weights
weights_median = np.median(class_weights)
weights = [weight / weights_median for weight in weights]

In [43]:
# 3. Train the XGBoost classifier with class weights
dtrain = xgb.DMatrix(X_tfidf_train, label=y_train, weight=weights)
dtest = xgb.DMatrix(X_tfidf_test, label=y_test)

params = {
    'objective': 'multi:softmax',
    'num_class': len(news_df['category'].unique()),
    'max_depth': 10,
    'silent': 1,
    'colsample_bytree': 0.7,
    'subsample': 0.7,
    'learning_rate': 0.05,
    'verbosity' : 0
}

watchlist = [(dtrain, 'train'), (dtest, 'eval')]
num_round = 1000

xgb_model = xgb.train(params, dtrain, num_round, watchlist, early_stopping_rounds=10,verbose_eval=0)

# 4. Evaluate the model
y_pred = xgb_model.predict(dtest)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

In [44]:
print(report)

                    precision    recall  f1-score   support

    ARTS & CULTURE       0.28      0.42      0.34       653
BUSINESS & FINANCE       0.40      0.47      0.43      1377
            COMEDY       0.15      0.24      0.19       929
             CRIME       0.25      0.48      0.33       566
    DIVERSE VOICES       0.49      0.28      0.36      2160
           DIVORCE       0.52      0.59      0.55       685
         EDUCATION       0.25      0.49      0.33       365
     ENTERTAINMENT       0.40      0.38      0.39      2955
       ENVIRONMENT       0.33      0.43      0.37       698
      FOOD & DRINK       0.57      0.62      0.59      1654
     HOME & LIVING       0.33      0.48      0.39       863
            IMPACT       0.20      0.25      0.22       617
             MEDIA       0.19      0.37      0.25       481
     MISCELLANEOUS       0.13      0.18      0.15       879
         PARENTING       0.57      0.52      0.55      2469
          POLITICS       0.79      0.47

#### BERT based solution found here: https://colab.research.google.com/drive/1lQnviwohO5Rs71pM_LNpEnzuAtdVuMkl#scrollTo=wENRYGOJuq7S 
#### It beats the XGBOOST by 23 percentage points. Please have a look at the link.

### --------------------------------------------------------------------------------------------------------------

### Unkown categories

You will next assume that the categories are unknown (e.g., you are NOT allowed to use the `category` attribute). However, you CAN assume that the number of categories is known.

Your solution should:
- Identify news headlines that belong to the same category
- Provide a human-understandable representation of each category

#### Solution: 
#### For this solution I use a FAISS based clustering with TFIDF features with 1000 max features due to memory contraints. 
#### THe cluster assignment i done with faiss, and the keyword extraction is done per cluster by taking top 100 words in each cluster and filtering them using a noun and dictionary filter.
#### The final word representations was a bit unclear and needs better feature representations, which can give better words at the end.

In [5]:
# TODO: provide your solution here.
# Convert headlines to TF-IDF vectors
headlines = news_df.headline.tolist()
headlines_preprocessed = list(map(preprocess_text, headlines))

In [41]:
vectorizer = TfidfVectorizer(max_df=0.85, stop_words='english', use_idf=True,max_features=1000)
tfidf_sparse_matrix = vectorizer.fit_transform(headlines_preprocessed)
feature_names = vectorizer.get_feature_names()

# #Use TruncatedSVD to reduce dimensionality of the sparse TF-IDF matrix
# n_components = 1000
# svd = TruncatedSVD(n_components=n_components)
# reduced_tfidf_matrix = svd.fit_transform(tfidf_sparse_matrix)

tfidf_sparse_matrix.shape

(209527, 1000)

In [42]:
# Convert the reduced TF-IDF matrix to float32 for FAISS
#reduced_tfidf_matrix = np.asarray(reduced_tfidf_matrix, dtype=np.float32)
tfidf_dense_matrix = tfidf_sparse_matrix.toarray()
tfidf_sparse_matrix = np.asarray(tfidf_dense_matrix, dtype=np.float32)
# Create a FAISS indexflatl2 for clustering
dimensionality = tfidf_dense_matrix.shape[1]
cluster_index = faiss.IndexFlatL2(dimensionality)

# Initialize the k-means clustering in FAISS
k = 25 # Number of clusters
niter = 50 # Number of iterations
faiss_clustering = faiss.Clustering(dimensionality, k)
faiss_clustering.niter = niter

# Train the clustering model
#faiss_clustering.train(reduced_tfidf_matrix, cluster_index)
faiss_clustering.train(tfidf_sparse_matrix, cluster_index)

# Assign each headline to a cluster
#_, cluster_assignments = cluster_index.search(reduced_tfidf_matrix, 1)
_, cluster_assignments = cluster_index.search(tfidf_sparse_matrix, 1)
cluster_assignments = cluster_assignments.flatten()


In [44]:
pd.DataFrame(cluster_assignments).value_counts()

19    188576
5       2116
1       2030
23      1860
21      1810
11      1722
14      1499
10      1364
3       1352
7        972
22       898
8        787
12       648
24       635
17       588
15       553
20       481
4        429
18       391
2        383
16       374
9         37
0         11
6          6
13         5
dtype: int64

In [49]:
feature_names = vectorizer.get_feature_names()

# Get the set of English words
english_words = set(words.words())

def filter_keywords(keywords):
    """Filter keywords to retain only dictionary nouns."""
    # Tokenize and get part-of-speech tags
    pos_tags = nltk.pos_tag(keywords)
    
    # Filter for nouns that are in the English dictionary
    filtered_keywords = [word for word, pos in pos_tags if (word in english_words) and (pos in ['NN', 'NNS'])]
    
    return filtered_keywords

def get_top_keywords(tfidf_matrix, cluster_assignments, cluster_number, top_n=100):
    """Get top N keywords for a given cluster."""
    cluster_data = tfidf_matrix[cluster_assignments == cluster_number]
    # Sum TF-IDF vectors of all samples in the cluster
    summed_cluster_tfidf = np.sum(cluster_data, axis=0)
    # Get indices of top N keywords
    top_indices = np.argsort(summed_cluster_tfidf)[-top_n:][::-1]
    top_indices = np.array(top_indices).flatten()
    # Get keywords
    keywords = [feature_names[idx] for idx in top_indices]
    # Filter the keywords to retain only dictionary nouns
    filtered_keywords = filter_keywords(keywords)
    return filtered_keywords[:top_n]

cluster_representations = {}
for cluster_num in range(25):
    keywords = get_top_keywords(reduced_tfidf_matrix, cluster_assignments, cluster_num)
    cluster_representations[cluster_num] = keywords


In [50]:
cluster_representations

{0: ['kind',
  'baby',
  'loss',
  'abortion',
  'mean',
  'meet',
  'issue',
  'joe',
  'hero',
  'season',
  'journalist',
  'lie',
  'launch',
  'study',
  'law',
  'record',
  'mayor',
  'egg',
  'sale',
  'restaurant',
  'learn',
  'fact',
  'depression',
  'person',
  'job',
  'man',
  'pregnancy',
  'pound',
  'ban',
  'royal',
  'spending',
  'soccer',
  'evolution',
  'future',
  'hospital',
  'drink',
  'price',
  'force',
  'freedom',
  'mind',
  'holiday',
  'scandal',
  'pick',
  'lawsuit',
  'program',
  'beauty',
  'strike',
  'drop',
  'murder',
  'party'],
 1: ['abortion',
  'animal',
  'album',
  'abuse',
  'address',
  'air',
  'autism',
  'art',
  'battle',
  'arrest',
  'bank',
  'advice',
  'benefit',
  'birthday',
  'artist',
  'award',
  'ben',
  'blood',
  'actress',
  'age',
  'book',
  'border',
  'college',
  'beat',
  'choice',
  'box',
  'breast',
  'director',
  'celebrity',
  'chief',
  'brand',
  'cup',
  'career',
  'body',
  'chicken',
  'face',
  'ci

## 2. Detecting complex query operations in natural language questions 

*You are building a pipeline that translates user-questions in corresponding SQL queries. As part of this pipeline, you need to build a classifier that detects complex query operations in a question.*

### Dataset

We provide an excerpt of the [Spider dataset](https://yale-lily.github.io/spider) containing around 1k pairs of natural language questions and their corresponding SQL query. It contains the following attributes:

- `question`: the natural language question.
- `query`: the SQL query corresponding to the question.
- `col_names`: description of the content of the columns
- `col_names_original`: names of the columns in the database schema
- `has_join`: if the query contains a join.
- `has_groupby`: if the query contains a "group by" operation.
- `has_orderby`: if the query contains an "order by" operation.

### Objectives

1) Choose **one** of the "join", "group by" or "order by" operators, and build a binary classifier that, given a natural language question, detects if the corresponding query will use this operator. You may NOT use the "query" field. 

> We expect you to explain and show that your solution works as expected (e.g., through metrics on a test dataset)

2) Describe in details how you would proceed if you had to identify which column from the schema (provided in the `col_names_original` in the dataset) needs to be put under the "group by" or "order by" statement. You can implement your solution but it is not mandatory

In [58]:
import pandas as pd
fname ="./data/queries.json"
queries_df = pd.read_json(fname)
queries_df.head()

,question,query,col_names,col_names_original,has_join,has_groupby,has_orderby
0,In which year were most departments established?,SELECT creation FROM department GROUP BY creat...,"[*, department id, name, creation, ranking, bu...","[*, Department_ID, Name, Creation, Ranking, Bu...",False,True,True
1,List the official name and status of the city ...,"SELECT Official_Name , Status FROM city ORDER...","[*, city id, official name, status, area km 2,...","[*, City_ID, Official_Name, Status, Area_km_2,...",False,False,True
2,What is the official name and status of the ci...,"SELECT Official_Name , Status FROM city ORDER...","[*, city id, official name, status, area km 2,...","[*, City_ID, Official_Name, Status, Area_km_2,...",False,False,True
3,Show the status of the city that has hosted th...,SELECT T1.Status FROM city AS T1 JOIN farm_com...,"[*, city id, official name, status, area km 2,...","[*, City_ID, Official_Name, Status, Area_km_2,...",True,True,True
4,What is the status of the city that has hosted...,SELECT T1.Status FROM city AS T1 JOIN farm_com...,"[*, city id, official name, status, area km 2,...","[*, City_ID, Official_Name, Status, Area_km_2,...",True,True,True


#### For the binary classification I use an XGBOOST model and we get an accuracy of 89%. As the categories are not very imbalanced category can be a valid metric to use.

In [76]:
# Splitting the dataset into training and test sets
train_df, test_df = train_test_split(queries_df, test_size=0.2, random_state=42)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.85, max_features=1000,ngram_range=(1,1))

# Fit and transform the questions in the training set
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['question'])

# Transform the questions in the test set
X_test_tfidf = tfidf_vectorizer.transform(test_df['question'])

# Extract labels for training and test set
y_train = train_df['has_groupby']
y_test = test_df['has_groupby']

X_train_tfidf.shape, X_test_tfidf.shape

((904, 898), (227, 898))

In [77]:
# Initialize Logistic Regression model
clf = LogisticRegression(random_state=42, max_iter=1500)

# Train the model
clf.fit(X_train_tfidf, y_train)

# Predict on test set
y_pred = clf.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(classification_rep)

              precision    recall  f1-score   support

       False       0.96      0.76      0.85        94
        True       0.85      0.98      0.91       133

    accuracy                           0.89       227
   macro avg       0.90      0.87      0.88       227
weighted avg       0.90      0.89      0.88       227



In [78]:
# # Perform 10-fold cross-validation
# cv_scores = cross_val_score(clf, X_train_tfidf, y_train, cv=10, scoring="accuracy")

# cv_scores.mean()

0.8773382173382174

### Detail solution for the column identification for group by and order by:

In [54]:
"""
1. Using traditional heuristic based techniques we could identify aggreggation keywords like "each", "per", "total" and oredring keywords like "sorted", "descending", etc. 
Then we can check if any of the original col names occur in the questions. We can try entity linking with aggregation and ordering keywords to to try and ascertain the solution. 
This solution is very flimsy as it needs a lot of manual annotation and word maintainability.
2. Ideally this problem can be solved by  text to text transfer where we take a model like T5 and finetune it in a pattern like 
"Question + original sql columns["A","B","C"] >> Group_by B order by C" . As T5 is a language model it cann generalize to other questions also.
3. Another method would be to gather a lot of text to sql datasets and train a T5 based model from scratch. The finetuning can be done with step 2 which will help it generalize better.
The dataset can be augmented by doing question generation from a second model which should be verified by human annotators.
3. The best possible current way is to use any sort of instruction tuned model, like LLAMA-2. As they have already been trained on text to sql datasets it is very simple to pass a prompt
to identify the group by column and the order by column.
"""

'\n1. Using traditional heuristic based techniques we could identify aggreggation keywords like "each", "per", "total" and oredring keywords like "sorted", "descending", etc. \nThen we can check if any of the original col names occur in the questions. We can try entity linking with aggregation and ordering keywords to to try and ascertain the solution. \nThis solution is very flimsy as it needs a lot of manual annotation and word maintainability.\n2. Ideally this problem can be solved by  text to text transfer where we take a model like T5 and finetune it in a pattern like \n"Question + original sql columns["A","B","C"] >> Group_by B order by C" . As T5 is a language model it cann generalize to other questions also.\n3. Another method would be to gather a lot of text to sql datasets and train a T5 based model from scratch. The finetuning can be done with step 2 which will help it generalize better.\nThe dataset can be augmented by doing question generation from a second model which sho